<a href="https://colab.research.google.com/github/Asif-Tanvir/Asif-Tanvir/blob/main/SmallCAP_Market_Cap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()


Saving Smallcap.ai - Company Details.csv to Smallcap.ai - Company Details.csv


In [3]:
!pip install duckduckgo_search
!pip install pandas
!pip install requests
!pip install beautifulsoup4
!pip install groq
!pip install json

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 9.8 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [4]:
!pip install yahooquery

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 37.3 MB/s eta 0:00:00
  Attempting uninstall: lxml
    Found existing installation: lxml 5.3.1
    Uninstalling lxml-5.3.1:
      Successfully uninstalled lxml-5.3.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
duckduckgo-search 7.5.2 requires lxml>=5.3.0, but you have lxml 4.9.4 which is incompatible.


In [ ]:
pip install yahooquery

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 66.5 MB/s eta 0:00:00
  Attempting uninstall: lxml
    Found existing installation: lxml 5.3.1
    Uninstalling lxml-5.3.1:
      Successfully uninstalled lxml-5.3.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
duckduckgo-search 7.5.2 requires lxml>=5.3.0, but you have lxml 4.9.4 which is incompatible.


In [ ]:
import pandas as pd
import time
import re
import requests
from yahooquery import Ticker
from bs4 import BeautifulSoup
import groq  # Using Groq API

# Set up your Groq API key
GROQ_API_KEY = "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"

# Initialize Groq client
client = groq.Groq(api_key=GROQ_API_KEY)

# Load CSV file
csv_path = "/content/Smallcap.ai - Company Details.csv"
companies_df = pd.read_csv(csv_path)

# Select rows to process (Example: Rows 723-850)
start_row, end_row = 4000, 4500
companies_df = companies_df.iloc[start_row:end_row].reset_index(drop=True)

print(f"Processing {len(companies_df)} companies (Rows {start_row}-{end_row})...")

# Function to fetch industry from Yahoo Finance
def get_industry_from_yahoo(company_name):
    try:
        ticker = Ticker(company_name)
        summary_profile = ticker.summary_profile

        # Extract industry or sector information
        for key, value in summary_profile.items():
            if "industry" in value:
                return value["industry"]
            elif "sector" in value:
                return value["sector"]

        return "Industry Not Found"

    except Exception as e:
        print(f"Error fetching industry for {company_name}: {e}")
        return "Error retrieving data"

# Function to scrape CAGR from market research websites
def scrape_cagr_from_web(company_name_name):
    try:
        # Search URL (Replace with trusted sources)
        search_url = f"https://www.statista.com/search/?q={company_name}+industry+CAGR"

        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
        }

        response = requests.get(search_url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")

        # Extract CAGR percentage from text
        text_content = soup.get_text(separator=" ", strip=True)
        match = re.search(r"(\d+\.?\d*)% CAGR", text_content)
        if match:
            return match.group(1) + "%"

        return "Not Found"

    except Exception as e:
        print(f"Error fetching CAGR for {company_name}: {e}")
        return "Error retrieving data"

# Function to query Groq AI for CAGR
def get_cagr_from_groq(company_name):
    try:
        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[
                {"role": "system", "content": "You are an AI assistant providing accurate industry CAGR data."},
                {"role": "user", "content": f"Tell me about the Industry growth rate of this {company_name}. Give accurate data with '%'. Dont need to write any description, only numbers"}
            ],
            max_tokens=200,
            temperature=0.5
        )

        ai_response = response.choices[0].message.content.strip()

        # Extract CAGR using regex
        match = re.search(r"(\d+\.?\d*)% CAGR", ai_response)
        if match:
            return match.group(1) + "%"

        return ai_response  # Return full response if no percentage found

    except Exception as e:
        print(f"Error fetching AI-based CAGR for {company_name}: {e}")
        return "Error retrieving data"

# Process company data
extracted_data = []

for index, row in companies_df.iterrows():
    company_name = row["Company Name"]
    print(f"Fetching Industry & CAGR for {company_name}...")

    # Get industry from Yahoo Finance
    company_name_name = get_industry_from_yahoo(company_name)

    # Get CAGR from Market Research Websites
    industry_cagr = scrape_cagr_from_web(company_name)

    # If CAGR not found, use Groq AI as a fallback
    if industry_cagr == "Not Found":
        industry_cagr = get_cagr_from_groq(company_name)

    extracted_data.append({
        "Company Name": company_name,

        "Industry Growth Rate (CAGR)": industry_cagr
    })

    time.sleep(5)  # Prevents hitting API rate limits

# Convert to DataFrame and save results
cagr_df = pd.DataFrame(extracted_data)
output_path_csv = "/content/industry_cagr_groq_4000_4500.csv"
cagr_df.to_csv(output_path_csv, index=False)

print(f"✅ Data extraction complete. Saved to: {output_path_csv}")


Processing 500 companies (Rows 4000-4500)...
Fetching Industry & CAGR for Elmira Savings Bank...
Fetching Industry & CAGR for Elmira Savings Bank...
Fetching Industry & CAGR for Elmira Savings Bank...
Fetching Industry & CAGR for Elrond (early-stage est.)...
Fetching Industry & CAGR for Elrond (early-stage est.)...
Fetching Industry & CAGR for Eltel Networks...
